In [274]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option('display.max_columns', None)


In [275]:
df = pd.read_csv('./nettiauto_dataset.csv')
# df = df.drop('color', axis=1)
df.head(25)

,make,model,color,driveType,price,totalOwners,kilometers,seats,power,batteryCapacity,age,isSuv,metallicColor,airconditioning,parking_sensors,satellite_navigator,cruise_control_adaptive,parking_camera_simple_camera,seat_heaters,driving_assistant,alloy_wheels,electric_mirrors,lane_departure_warning_system,electrically_operated_tailgate,leather_upholstery,emergency_brake_assist,collision_avoidance_system,cruise_control_traditional,parking_assistant,tow_bar,electric_seats_with_memory,heated_steering_wheel,sunroof,sun_hatch_with_panorama,adaptive_headlights,sport_seats,fuel_battery_powered_heater,parking_camera_360-degree_camera,battery_preheating,electric_seats_without_memory,air_suspension,curve_lights,head_up_display,sport_base
0,porsche,taycan,black,rwd,101880.0,1,2100.0,5,300.0,79.00,2.416667,0,0,1,1,1,0,0,1,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
1,mercedes-benz,eqs,gray,4wd,125900.0,1,30000.0,5,385.0,108.40,2.000000,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,1,1,1,1,0,1,0,0,0
2,nissan,leaf,black,fwd,35800.0,1,10000.0,5,160.0,62.00,2.000000,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,citroen,e-c4,blue,fwd,33800.0,1,17000.0,5,100.0,50.00,2.000000,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,hyundai,kona,white,fwd,27590.0,1,56000.0,5,150.0,64.00,2.750000,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,1,0
5,citroen,e-c4,beige,fwd,29700.0,1,34000.0,5,100.0,50.00,2.000000,0,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,polestar,2,black,4wd,60780.0,1,7000.0,5,300.0,75.00,2.000000,0,0,1,1,1,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,1,0,1,1,1,1,0,0,0,1,0,0
7,peugeot,e-2008,orange,fwd,25790.0,1,13000.0,5,100.0,50.00,2.500000,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,bmw,ix3,white,rwd,51800.0,1,17000.0,5,210.0,80.00,2.166667,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
9,bmw,ix3,white,rwd,49900.0,1,25500.0,5,210.0,80.00,2.000000,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0


In [276]:
# categorical values
df_encoded = pd.get_dummies(df, columns=[
    "make",
    "model",
    "color",
    "driveType"
])
df_encoded.head(25)


,price,totalOwners,kilometers,seats,power,batteryCapacity,age,isSuv,metallicColor,airconditioning,parking_sensors,satellite_navigator,cruise_control_adaptive,parking_camera_simple_camera,seat_heaters,driving_assistant,alloy_wheels,electric_mirrors,lane_departure_warning_system,electrically_operated_tailgate,leather_upholstery,emergency_brake_assist,collision_avoidance_system,cruise_control_traditional,parking_assistant,tow_bar,electric_seats_with_memory,heated_steering_wheel,sunroof,sun_hatch_with_panorama,adaptive_headlights,sport_seats,fuel_battery_powered_heater,parking_camera_360-degree_camera,battery_preheating,electric_seats_without_memory,air_suspension,curve_lights,head_up_display,sport_base,make_audi,make_bmw,make_byd,make_citroen,make_cupra,make_dacia,make_ds,make_fiat,make_ford,make_hyundai,make_jaguar,make_kia,make_lexus,make_maxus,make_mazda,make_mercedes-benz,make_mg,make_mini,make_muu merkki,make_nissan,make_opel,make_peugeot,make_polestar,make_porsche,make_renault,make_seat,make_skoda,make_subaru,make_tesla,make_toyota,make_volkswagen,make_volvo,model_2,model_208,model_3,model_500e,model_ariya,model_atto3,model_born,model_bz4x,model_c4,model_c40,model_citigo,model_combo-e,model_cooper,model_corsa-e,model_e-2008,model_e-208,model_e-308,model_e-berlingo,model_e-c4,model_e-c4 x,model_e-niro,model_e-soul,model_e-spacetourer,model_e-traveller,model_e-tron,model_e-tron gt,model_e-tron gt rs,model_e-up!,model_enyaq,model_eqa,model_eqb,model_eqc,model_eqe,model_eqs,model_eqs suv,model_eqv,model_euniq,model_ev6,model_ev9,model_han,model_i-pace,model_i3,model_i3s,model_i4,model_i4 m50,model_i7,model_id. buzz,model_id3,model_id4,model_id5,model_ioniq 5,model_ioniq 6,model_ioniq electric,model_ix,model_ix m60,model_ix1,model_ix3,model_kona,model_kona electric,model_leaf,model_marvel r,model_megane,model_mg4,model_mg5,model_mifa 9,model_mii electric,model_model 3,model_model s,model_model x,model_model y,model_mokka,model_mokka-e,model_mustang,model_mustang mach-e,model_muu malli,model_mx-30,model_niro,model_proace city verso ev,model_proace verso ev,model_q4 e-tron,model_q8 e-tron,model_solterra,model_spring,model_t90,model_tang,model_taycan,model_twingo,model_up!,model_ux,model_xc40,model_zafira-e,model_zoe,model_zs,color_beige,color_black,color_blue,color_brown,color_copper,color_gray,color_green,color_orange,color_other,color_red,color_silver,color_turquoise,color_violet,color_white,color_yellow,driveType_4wd,driveType_fwd,driveType_rwd
0,101880.0,1,2100.0,5,300.0,79.00,2.416667,0,0,1,1,1,0,0,1,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,125900.0,1,30000.0,5,385.0,108.40,2.000000,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,0,0,1,1,1,1,1,1,0,1,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,Fa

In [277]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop('price', axis=1)
y = df_encoded['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape

((3184, 182), (796, 182))

In [278]:
import xgboost as xgb

xgb_regressor = xgb.XGBRegressor(objective ='reg:squarederror',
                                 n_estimators=100,
                                 max_depth=8,
                                 reg_lambda=1, # näitä voi säädellä muttei kauheesti oo vaikuttanut malliin
                                 reg_alpha=0,# näitä voi säädellä muttei kauheesti oo vaikuttanut malliin
                                 num_parallel_tree=5, # random forest 🤷‍♂️
)

xgb_regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=5, predictor=None,
             random_state=None, ...)

In [279]:
y_pred = xgb_regressor.predict(X_test)


In [280]:
mean_absolute_error(y_test, y_pred)

3235.212186420383

In [281]:
mean_squared_error(y_test, y_pred)

27628104.553577542

In [283]:
r2_score(y_test, y_pred)

0.9620857596897376

In [315]:
def predict_make_model(make, model):
    m = (X_test[f'make_{make}'] == 1) & (X_test[f'model_{model}'] == 1)
    X_id3 = X_test[m]
    y_id3 = y_test[m]

    y_id3_pred = xgb_regressor.predict(X_id3)

    print('mean_absolute_error: {}'.format(mean_absolute_error(y_id3, y_id3_pred)))
    print('r2_score: {}'.format(r2_score(y_id3, y_id3_pred)))
    print()
    for real, pred in zip(y_id3, y_id3_pred):
        print(f'{real} -> {pred} ~ {real - pred} <> {pred / real}')
    
predict_make_model('volkswagen', 'id3')

mean_absolute_error: 2270.527276400862
r2_score: 0.8268803557949724

48500.0 -> 42559.08203125 ~ 5940.91796875 <> 0.8775068460051546
46415.0 -> 47093.69140625 ~ -678.69140625 <> 1.0146222429440914
44900.0 -> 46959.1015625 ~ -2059.1015625 <> 1.0458597229955457
48990.0 -> 43281.84765625 ~ 5708.15234375 <> 0.8834833161104307
51355.0 -> 52385.84765625 ~ -1030.84765625 <> 1.0200729754892415
42900.0 -> 39357.796875 ~ 3542.203125 <> 0.9174311625874126
45940.0 -> 47491.97265625 ~ -1551.97265625 <> 1.0337826002666521
41000.0 -> 46714.2734375 ~ -5714.2734375 <> 1.1393725228658536
30900.0 -> 36901.29296875 ~ -6001.29296875 <> 1.1942166009304207
51965.0 -> 49623.6015625 ~ 2341.3984375 <> 0.9549427799961513
39900.0 -> 41678.4765625 ~ -1778.4765625 <> 1.0445733474310777
41900.0 -> 46755.72265625 ~ -4855.72265625 <> 1.1158883688842482
42500.0 -> 43300.99609375 ~ -800.99609375 <> 1.0188469669117648
37900.0 -> 38157.58984375 ~ -257.58984375 <> 1.006796565798153
37900.0 -> 35302.27734375 ~ 2597.72265625

In [316]:
predict_make_model('bmw', 'ix1')

mean_absolute_error: 4223.40185546875
r2_score: 0.3560024687907284

71580.0 -> 71592.6796875 ~ -12.6796875 <> 1.0001771400880135
56900.0 -> 62007.0390625 ~ -5107.0390625 <> 1.0897546408172232
66900.0 -> 68991.15625 ~ -2091.15625 <> 1.0312579409566518
69900.0 -> 65576.7109375 ~ 4323.2890625 <> 0.9381503710658083
78290.0 -> 69195.5625 ~ 9094.4375 <> 0.8838365372333631
62750.0 -> 69502.984375 ~ -6752.984375 <> 1.107617280876494
62490.0 -> 64506.68359375 ~ -2016.68359375 <> 1.0322721010361657
62900.0 -> 67288.9453125 ~ -4388.9453125 <> 1.0697765550476948


In [318]:
predict_make_model('tesla', 'model 3')

mean_absolute_error: 2145.4755108173076
r2_score: 0.6439320818833593

44900.0 -> 45650.09765625 ~ -750.09765625 <> 1.016705961163697
47800.0 -> 51797.41015625 ~ -3997.41015625 <> 1.083627827536611
39880.0 -> 42759.58984375 ~ -2879.58984375 <> 1.072206365189318
52000.0 -> 43380.91796875 ~ 8619.08203125 <> 0.8342484224759615
38900.0 -> 42363.671875 ~ -3463.671875 <> 1.0890404080976863
36900.0 -> 35981.203125 ~ 918.796875 <> 0.9751003556910569
36890.0 -> 40282.6875 ~ -3392.6875 <> 1.0919676741664408
40850.0 -> 43794.2578125 ~ -2944.2578125 <> 1.0720748546511627
43370.0 -> 44799.90234375 ~ -1429.90234375 <> 1.0329698488298362
45000.0 -> 48293.90625 ~ -3293.90625 <> 1.0731979166666668
48490.0 -> 51630.05859375 ~ -3140.05859375 <> 1.06475682808311
39890.0 -> 40083.0859375 ~ -193.0859375 <> 1.0048404597016796
49880.0 -> 49810.80859375 ~ 69.19140625 <> 0.998612842697474
42800.0 -> 43538.65234375 ~ -738.65234375 <> 1.0172582323306074
47000.0 -> 46361.2890625 ~ 638.7109375 <> 0.9864104055851064


In [325]:
predict_make_model('opel', 'mokka-e')

mean_absolute_error: 1777.0099158653845
r2_score: 0.5571670690165371

38570.0 -> 39112.00390625 ~ -542.00390625 <> 1.0140524735869847
38080.0 -> 39112.00390625 ~ -1032.00390625 <> 1.027100942916229
38900.0 -> 40646.06640625 ~ -1746.06640625 <> 1.0448860258676091
38960.0 -> 40646.06640625 ~ -1686.06640625 <> 1.0432768584766428
32990.0 -> 35425.62109375 ~ -2435.62109375 <> 1.0738290722567445
43700.0 -> 43052.4921875 ~ 647.5078125 <> 0.9851828875858124
42900.0 -> 39515.15625 ~ 3384.84375 <> 0.9210992132867133
44870.0 -> 42728.21484375 ~ 2141.78515625 <> 0.9522668786215734
39900.0 -> 36334.94140625 ~ 3565.05859375 <> 0.9106501605576441
36990.0 -> 39112.00390625 ~ -2122.00390625 <> 1.0573669615098675
36990.0 -> 37216.75 ~ -226.75 <> 1.0061300351446336
41900.0 -> 39031.875 ~ 2868.125 <> 0.9315483293556086
37700.0 -> 38403.29296875 ~ -703.29296875 <> 1.0186549859084881


In [326]:
predict_make_model('audi', 'e-tron')

mean_absolute_error: 6234.279854910715
r2_score: 0.5015313269014482

49800.0 -> 58210.3984375 ~ -8410.3984375 <> 1.1688835027610442
69900.0 -> 73602.84375 ~ -3702.84375 <> 1.0529734442060086
43900.0 -> 46943.6484375 ~ -3043.6484375 <> 1.0693313994874716
69500.0 -> 62805.06640625 ~ 6694.93359375 <> 0.9036700202338129
59990.0 -> 66008.7421875 ~ -6018.7421875 <> 1.1003290913068844
67900.0 -> 62040.65234375 ~ 5859.34765625 <> 0.9137062200846834
79700.0 -> 97517.515625 ~ -17817.515625 <> 1.223557285131744
47900.0 -> 54013.55859375 ~ -6113.55859375 <> 1.1276317034185803
79900.0 -> 84948.0 ~ -5048.0 <> 1.0631789737171464
68000.0 -> 65683.25 ~ 2316.75 <> 0.9659301470588235
67800.0 -> 64964.75390625 ~ 2835.24609375 <> 0.9581822110066371
62800.0 -> 65879.328125 ~ -3079.328125 <> 1.0490338873407643
68490.0 -> 62302.33203125 ~ 6187.66796875 <> 0.909655891827274
64900.0 -> 75051.9375 ~ -10151.9375 <> 1.1564243066255777
